# 유사한 단어 찾기 게임

1. 사전 학습된 모델 또는 적절한 데이터셋을 찾는다
2. 워드 임베딩 모델을 학습시킨다.
3. 단어 유사도가 0.8 이상인 A,B를 랜덤 추출한다
4. A,B와 대응되는 C를 추출한다.
5. D를 입력 받는다.

=>
A:B = C:D 관계에 대응하는 D를 찾는 게임을 만든다. <br>
ex) A : 산, B : 바다, C : 나무, D :  물

**<출력 예시>**

관계 [ 수긍 : 추락 = 대사관 : ? ] <br>
모델이 예측한 가장 적합한 단어: 잠입 <br>
당신의 답변과 모델 예측의 유사도: 0.34 <br>
아쉽네요. 더 생각해보세요.

In [ ]:
import random
from gensim.models import FastText
import re

print("대용량 FastText 모델('cc.ko.300.bin') 로딩을 시작합니다.")
print("이 과정은 수 분 이상 소요될 수 있습니다.")

try:
    model = FastText.load_fasttext_format('cc.ko.300.bin')
except FileNotFoundError:
    print("\n[오류] 'cc.ko.300.bin' 파일을 찾을 수 없습니다.")
    exit()

print("\n모델 로딩 완료!")
print("정제된 명사 목록 파일('filtered_noun_list.txt')을 불러옵니다...")

try:
    with open('filtered_noun_list.txt', 'r', encoding='utf-8') as f:
        filtered_vocab = []
        for line in f:
            filtered_vocab.append(line.strip())
    print(f"목록 로딩 완료! 총 {len(filtered_vocab):,}개의 단어를 사용합니다.")
except FileNotFoundError:
    print("\n[오류] 'filtered_noun_list.txt' 파일을 찾을 수 없습니다.")
    exit()

print("-" * 30)
print("게임을 시작합니다.")

while True:
    a, b, c, predicted_d = None, None, None, None
    retries = 0
    while not all([a, b, c, predicted_d]) and retries < 100:
        retries += 1
        try:
            a = random.choice(filtered_vocab)
            similar_to_a = model.wv.most_similar(a, topn=50)

            candidates_c = []
            for word, sim in similar_to_a:
                if sim > 0.5 and word in filtered_vocab and word != a:
                    candidates_c.append(word)
            
            if not candidates_c:
                continue
            c = random.choice(candidates_c)

            candidates_b = []
            for word, sim in similar_to_a:
                if sim > 0.6 and word in filtered_vocab and word != a and word != c:
                    candidates_b.append(word)

            if not candidates_b:
                continue
            b = random.choice(candidates_b)

            top_predictions = model.wv.most_similar(positive=[c, b], negative=[a], topn=10)
            
            for word, _ in top_predictions:
                if word in filtered_vocab:
                    predicted_d = word
                    break
            
            if predicted_d is None and top_predictions:
                predicted_d = top_predictions[0][0]

        except (KeyError, IndexError):
            continue

    if not predicted_d:
        print("적절한 문제를 만드는 데 실패했습니다. 다시 시도합니다.")
        continue

    print(f"\n관계 [ {a} : {b} = {c} : ? ]")
    
    user_d = input("정답이라고 생각하는 단어를 입력하세요: ")

    if user_d not in model.wv:
        print(f"'{user_d}'는 모델이 알지 못하는 단어입니다. 다른 단어를 입력해주세요.")
        continue

    similarity = model.wv.similarity(user_d, predicted_d)
    print(f"모델이 예측한 1순위 단어: {predicted_d}")
    print(f"당신의 답변과 1순위 예측의 유사도: {similarity:.2f}")

    if similarity > 0.6:
        print("정답입니다! 🎉 1순위 예측과 매우 유사하네요!")
    elif similarity > 0.3:
        print("비슷하네요! 조금 더 생각해볼까요? 🤔")
    else:
        print("아쉽네요. 더 생각해보세요. 😥")
    
    print("-" * 30)

대용량 FastText 모델('cc.ko.300.bin') 로딩을 시작합니다.
이 과정은 수 분 이상 소요될 수 있습니다.


C:\Users\Playdata\AppData\Local\Temp\ipykernel_896\1764106501.py:9: DeprecationWarning: Call to deprecated `load_fasttext_format` (use load_facebook_vectors (to use pretrained embeddings) or load_facebook_model (to continue training with the loaded full model, more RAM) instead).
  model = FastText.load_fasttext_format('cc.ko.300.bin')



모델 로딩 완료!
정제된 명사 목록 파일('filtered_noun_list.txt')을 불러옵니다...
목록 로딩 완료! 총 1,501,657개의 단어를 사용합니다.
------------------------------
게임을 시작합니다.

관계 [ 주창선 : 김사성 = 김유강 : ? ]
모델이 예측한 1순위 단어: 민병오
당신의 답변과 1순위 예측의 유사도: 0.00
아쉽네요. 더 생각해보세요. 😥
------------------------------

관계 [ 이수건설 : 선창산업 = 롯데자산개발 : ? ]
모델이 예측한 1순위 단어: 삼립식품
당신의 답변과 1순위 예측의 유사도: 0.13
아쉽네요. 더 생각해보세요. 😥
------------------------------

관계 [ 극락으로 : 저승으로 = 발홀로 : ? ]
모델이 예측한 1순위 단어: 연경으로
당신의 답변과 1순위 예측의 유사도: 0.08
아쉽네요. 더 생각해보세요. 😥
------------------------------

관계 [ 옆에있고 : 길건너에 = 캐널시티까지 : ? ]
모델이 예측한 1순위 단어: 캐널시티에서
당신의 답변과 1순위 예측의 유사도: 0.21
아쉽네요. 더 생각해보세요. 😥
------------------------------

관계 [ 과세표준은 : 매입세액의 = 종합소득세를 : ? ]
모델이 예측한 1순위 단어: 종합소득세의
당신의 답변과 1순위 예측의 유사도: 0.08
아쉽네요. 더 생각해보세요. 😥
------------------------------

관계 [ 유엔인권이사회에 : 인권이사회와 = 인권이사회는 : ? ]
